In [7]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import re
from collections import defaultdict
from data.scorer import score_submission, print_confusion_matrix, score_defaults, SCORE_REPORT
from nltk import word_tokenize
from scipy import sparse
from scipy.spatial.distance import cosine  # TODO may have to implement own maths formulas
from scipy.stats import entropy  # (kl-divergence) TODO may have to implement own maths formulas
from sklearn.linear_model import LogisticRegression  # TODO implement own classifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  # TODO implement own tokenizer + count + tf-idf
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder  # TODO implement my own mapping
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
# Load data from CSV
f_bodies = open('data/train_bodies.csv', 'r', encoding='utf-8')
csv_bodies = csv.DictReader(f_bodies)
bodies = []
for row in csv_bodies:
    body_id = int(row['Body ID'])
    if (body_id + 1) > len(bodies):
        bodies += [None] * (body_id + 1 - len(bodies))
    bodies[body_id] = row['articleBody']
f_bodies.close()
body_inverse_index = {bodies[i]: i for i in range(len(bodies))}

all_unrelated, all_discuss, all_agree, all_disagree = [], [], [], []  # each article = (headline, body, stance)

f_stances = open('data/train_stances.csv', 'r', encoding='utf-8')
csv_stances = csv.DictReader(f_stances)
for row in csv_stances:
    body = bodies[int(row['Body ID'])]
    if row['Stance'] == 'unrelated':
        all_unrelated.append((row['Headline'], body, row['Stance']))
    elif row['Stance'] == 'discuss':
        all_discuss.append((row['Headline'], body, row['Stance']))
    elif row['Stance'] == 'agree':
        all_agree.append((row['Headline'], body, row['Stance']))
    elif row['Stance'] == 'disagree':
        all_disagree.append((row['Headline'], body, row['Stance']))
f_stances.close()

In [3]:
# Split to train and validation 9:1
print('\tUnrltd\tDiscuss\t Agree\tDisagree')
print('All\t', len(all_unrelated), '\t', len(all_discuss), '\t', len(all_agree), '\t', len(all_disagree))

train_unrelated = all_unrelated[:len(all_unrelated) * 9 // 10]
train_discuss = all_discuss[:len(all_discuss) * 9 // 10]
train_agree = all_agree[:len(all_agree) * 9 // 10]
train_disagree = all_disagree[:len(all_disagree) * 9 // 10]

val_unrelated = all_unrelated[len(all_unrelated) * 9 // 10:]
val_discuss = all_discuss[len(all_discuss) * 9 // 10:]
val_agree = all_agree[len(all_agree) * 9 // 10:]
val_disagree = all_disagree[len(all_disagree) * 9 // 10:]

print('Train\t', len(train_unrelated), '\t', len(train_discuss), '\t', len(train_agree), '\t', len(train_disagree))
print('Valid.\t', len(val_unrelated), '\t', len(val_discuss), '\t', len(val_agree), '\t', len(val_disagree))

	Unrltd	Discuss	 Agree	Disagree
All	 36545 	 8909 	 3678 	 840
Train	 32890 	 8018 	 3310 	 756
Valid.	 3655 	 891 	 368 	 84


In [4]:
train_all = train_unrelated + train_discuss + train_agree + train_disagree  # each article = (headline, body, stance)
random.Random(0).shuffle(train_all)
train_all = np.array(train_all)

val_all = val_unrelated + val_discuss + val_agree + val_disagree
random.Random(0).shuffle(val_all)
val_all = np.array(val_all)

print('Train (Total)', train_all.shape, '\tValidation (Total)', val_all.shape)
print(np.count_nonzero(train_all[:, 2] == 'unrelated'), '\t',
      np.count_nonzero(train_all[:, 2] == 'discuss'), '\t',
      np.count_nonzero(train_all[:, 2] == 'agree'), '\t',
      np.count_nonzero(train_all[:, 2] == 'disagree'))
print(np.count_nonzero(val_all[:, 2] == 'unrelated'), '\t',
      np.count_nonzero(val_all[:, 2] == 'discuss'), '\t',
      np.count_nonzero(val_all[:, 2] == 'agree'), '\t',
      np.count_nonzero(val_all[:, 2] == 'disagree'))

Train (Total) (44974, 3) 	Validation (Total) (4998, 3)
32890 	 8018 	 3310 	 756
3655 	 891 	 368 	 84


In [5]:
pattern = re.compile("[^a-zA-Z0-9 ]+")
def tokenise(text):
    text = pattern.sub('', text.replace('\n', ' ').replace('-', ' ').lower())
    return word_tokenize(text)

for i in range(9):
    print(train_all[i, 0], tokenise(train_all[i, 0]))
print(train_all[0, 1], tokenise(train_all[0, 1]))

Al Qaida hostage killed in raid ['al', 'qaida', 'hostage', 'killed', 'in', 'raid']
UPDATE: Internet report of Ebola outbreak in Purdon not true ['update', 'internet', 'report', 'of', 'ebola', 'outbreak', 'in', 'purdon', 'not', 'true']
Christian Bale Won't Play Steve Jobs After All ['christian', 'bale', 'wont', 'play', 'steve', 'jobs', 'after', 'all']
#RIPFidel fuels false rumors of Cuban leader's death after another Fidel Castro dies ['ripfidel', 'fuels', 'false', 'rumors', 'of', 'cuban', 'leaders', 'death', 'after', 'another', 'fidel', 'castro', 'dies']
Boko Haram claims to have German hostage, denies ceasefire ['boko', 'haram', 'claims', 'to', 'have', 'german', 'hostage', 'denies', 'ceasefire']
Batmobile Stolen From 'Batman V Superman: Dawn Of Justice' Set? Report Claims That One Of The Batmobile Models Is Missing! Detroit Locals Responsible? ['batmobile', 'stolen', 'from', 'batman', 'v', 'superman', 'dawn', 'of', 'justice', 'set', 'report', 'claims', 'that', 'one', 'of', 'the', 'bat

In [8]:
def doc_to_tf(text):
    words = tokenise(text)
    ret = defaultdict(float)
    for word in words:
        ret[word] += 1.0
    return ret
    
for i in range(9):
    print(train_all[i, 0], doc_to_tf(train_all[i, 0]))
print(train_all[0, 1], doc_to_tf(train_all[0, 1]))

Al Qaida hostage killed in raid defaultdict(<class 'float'>, {'al': 1.0, 'qaida': 1.0, 'hostage': 1.0, 'killed': 1.0, 'in': 1.0, 'raid': 1.0})
UPDATE: Internet report of Ebola outbreak in Purdon not true defaultdict(<class 'float'>, {'update': 1.0, 'internet': 1.0, 'report': 1.0, 'of': 1.0, 'ebola': 1.0, 'outbreak': 1.0, 'in': 1.0, 'purdon': 1.0, 'not': 1.0, 'true': 1.0})
Christian Bale Won't Play Steve Jobs After All defaultdict(<class 'float'>, {'christian': 1.0, 'bale': 1.0, 'wont': 1.0, 'play': 1.0, 'steve': 1.0, 'jobs': 1.0, 'after': 1.0, 'all': 1.0})
#RIPFidel fuels false rumors of Cuban leader's death after another Fidel Castro dies defaultdict(<class 'float'>, {'ripfidel': 1.0, 'fuels': 1.0, 'false': 1.0, 'rumors': 1.0, 'of': 1.0, 'cuban': 1.0, 'leaders': 1.0, 'death': 1.0, 'after': 1.0, 'another': 1.0, 'fidel': 1.0, 'castro': 1.0, 'dies': 1.0})
Boko Haram claims to have German hostage, denies ceasefire defaultdict(<class 'float'>, {'boko': 1.0, 'haram': 1.0, 'claims': 1.0, 'to

In [9]:
# Build corpus of article bodies and headlines in training dataset
corpus = np.r_[train_all[:, 1], train_all[:, 0]]  # 0 to 44973 are bodies, 44974 to 89943 are headlines

print(corpus[0])
print(corpus[44974])

An attorney representing a man who says he realized he had inadvertently recorded audio of the Michael Brown shooting has provided that audio to CNN:

Attorney Lopa Blumenthal says the recording was made during a video chat. The audio captures what seem like 10 gunshot sounds—an initial group of six, then a pause, then four more. An autopsy of Brown's body performed at his family's request by an experienced forensic pathologist named Michael Baden indicated Brown was hit by at least six shots.

Eyewitness Dorian Johnson, a friend of Brown's, has said that officer Darren Wilson fired several shots at Brown after Brown had turned toward him and raised his hands. The Baden autopsy indicated that Brown had been shot exclusively in the front of his body.

Blumenthal says the man who made the recording has spoken to the FBI.

Read more on Ferguson.
Al Qaida hostage killed in raid


In [12]:
# Learn idf of every word in the corpus
df = defaultdict(float)
for doc in tqdm(corpus):
    words = tokenise(doc)
    seen = set()
    for word in words:
        if word not in seen:
            df[word] += 1.0
            seen.add(word)

print(list(df.items())[:10])

100%|███████████████████████████████████████████████████████████████████████████| 89948/89948 [01:58<00:00, 758.75it/s]


[('an', 32525.0), ('attorney', 1738.0), ('representing', 417.0), ('a', 48652.0), ('man', 13433.0), ('who', 28184.0), ('says', 11259.0), ('he', 29818.0), ('realized', 518.0), ('had', 26590.0)]


In [14]:
num_docs = corpus.shape[0]
idf = defaultdict(float)
for word, val in tqdm(df.items()):
    idf[word] = np.log((1.0 + num_docs) / (1.0 + val)) + 1.0  # smoothed idf

print(list(idf.items())[:10])

100%|████████████████████████████████████████████████████████████████████████| 24569/24569 [00:00<00:00, 386671.38it/s]


[('an', 2.017203073541107), ('attorney', 4.945932607675217), ('representing', 6.371516689504744), ('a', 1.6145293713628024), ('man', 2.90145403620269), ('who', 2.1604529215138246), ('says', 3.077986220335818), ('he', 2.1040970688136422), ('realized', 6.155094238863611), ('had', 2.2186700303762477)]


In [ ]:
# Load GLoVe word vectors
f_glove = open("data/glove.6B.50d.txt", "rb")  # download from https://nlp.stanford.edu/projects/glove/
# also try if higher-dimension/higher-vocabulary GLoVe vectors perform better
glove_vectors = {}
for line in tqdm(f_glove):
    glove_vectors[str(line.split()[0]).split("'")[1]] = np.array(list(map(float, line.split()[1:])))
# for key, value in glove_vectors.items():
#    print(key, value)
#    break
print(glove_vectors['glove'])

In [ ]:
# Convert a document to GloVe vectors, by computing tf-idf of each word * GLoVe of word / total tf-idf for document
def doc_to_glove(doc):
    doc_tf_idf = tf_idf_vec.transform([doc])
    # print(doc_tf_idf[:10])
    _, cols = doc_tf_idf[0].nonzero()
    doc_vector = np.array([0.0]*50)
    sum_tf_idf = 0
    for col in cols:
        word = col_to_word[col]
        if word in glove_vectors:
            # print(word, tf_idf[row, col], glove_vectors[word])
            doc_vector += glove_vectors[word] * doc_tf_idf[0, col]
            sum_tf_idf += doc_tf_idf[0, col]
    doc_vector /= sum_tf_idf
    return doc_vector
print(train_all[0,0])
print(doc_to_glove(train_all[0,0]))

In [ ]:
# Compute cosine similarity of GLoVe vectors for all headline-body pairs
def cosine_similarity(doc):
    return 1.0 - cosine(doc_to_glove(doc[0]), doc_to_glove(doc[1]))

for i in range(10):
    # unrelated should have lower than rest
    print(cosine_similarity(train_all[i]), train_all[i, 2])

In [ ]:
# Compute the KL-Divergence of language model (LM) representations of the headline and the body
eps = 0.1  # add a small value for every common word in the LM, as KL-divergence won't work if there are 0 values
def kl_divergence(doc):
    # Convert body and headline to bag of words representations
    vec = CountVectorizer(stop_words='english')
    vec_all = vec.fit_transform([doc[0], doc[1]])
    vec_headline = np.squeeze(np.array(vec_all[0].todense()))
    vec_body = np.squeeze(np.array(vec_all[1].todense()))
    
    # Compute a simple unigram LM of headline and body using bag of words / no. of words in doc
    lm_headline = (vec_headline + eps) / np.sum(vec_headline)
    lm_body = (vec_body + eps) / np.sum(vec_body)
    
    # Return KL-divergence of lm
    return entropy(lm_headline, lm_body)

for i in range(10):
    # unrelated should have lower than rest
    print(cosine_similarity(train_all[i]), train_all[i, 2])

In [ ]:
# Other feature 1 - distinguish unrelated vs rest
def ngram_match(doc):
    # Returns how many times n-grams (up to 3-gram) that occur in the article's headline occur on the article's body.
    vec = CountVectorizer(stop_words='english', ngram_range=(1, 3))
    vec.fit([doc[0]])
    vec_body = vec.transform([doc[1]])
    return np.power((np.sum(vec_body) / len(doc[1])), 1 / np.e)
    
for i in range(10):
    # unrelated should have lower than rest
    print(ngram_match(train_all[i]), train_all[i, 2])

In [ ]:
# Other feature 2 - distinguish discuss vs agree/disagree
def body_polarity(doc):
    # TODO (should be lower for discuss compared to agree/disagree)
    pass

In [ ]:
# Other feature 3 - distinguish agree vs disagree
sent_analyser = SentimentIntensityAnalyzer()

def sentiment_mismatch(doc):
    return np.absolute(sent_analyser.polarity_scores(doc[0])['compound'] - sent_analyser.polarity_scores(doc[1])['compound'])
# sentiment_mismatch doesn't do anything atm.. still predicts everything as discuss
# Values seem to be random (agree not neccesarily lower, disagree not neccesarily higher)
# Maybe use a feature based on no. of negative words similar to 'no's in the body? - more likely to be disagree
    
train_related = train_all[train_all[:, 2] == 'discuss']
for i in range(5):
    print(sentiment_mismatch(train_related[i]), train_related[i, 2])
train_agree = train_all[train_all[:, 2] == 'agree']
for i in range(5):
    # Should be lower
    print(sentiment_mismatch(train_agree[i]), train_agree[i, 2])
train_disagree = train_all[train_all[:, 2] == 'disagree']
for i in range(5):
    # Should be higher
    print(sentiment_mismatch(train_disagree[i]), train_disagree[i, 2])

In [ ]:
# Define function to convert (headline, body) to feature vectors for each document
ftrs = [cosine_similarity, kl_divergence, ngram_match]
def to_feature_array(doc):
    vec = np.array([0.0] * len(ftrs))
    for i in range(len(ftrs)):
        vec[i] = ftrs[i](doc)
    return vec

# Initialise x and y for train dataset
x_train = np.array([to_feature_array(doc) for doc in tqdm(train_all)])
print(x_train[:5])

le = LabelEncoder()
y_train = le.fit_transform(list(train_all[:, 2]))
print(y_train[:5])

In [ ]:
# Plot GLoVe distance vs KL-Divergence on a coloured scatter plot with different colours for each label
colours = np.array(['g', 'r', 'b', 'y'])
plt.scatter(list(x_train[:, 0]), list(x_train[:, 1]), c=colours[y_train])
plt.xlabel('Cosine Similarity of GLoVe vectors')
plt.ylabel('KL Divergence of Unigram LMs')
print([(colours[i], le.classes_[i]) for i in range(len(le.classes_))])
plt.show()

In [ ]:
# Initialise x and y for validation dataset
x_val = np.array([to_feature_array(doc) for doc in tqdm(val_all)])
print(x_val[:5])

In [ ]:
# TODO implement linear/logistic regression classifier using these features. Optimise params on validation set
clf = LogisticRegression(C=1e2, random_state=0, multi_class='multinomial', solver='saga')
# clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)  # TODO temporary classifier
clf.fit(x_train, y_train)

In [ ]:
# Predict y for validation set
y_pred = clf.predict(x_val)
print(y_pred[:5])

In [ ]:
# Prepare dataset format for score_submission in scorer.py
predicted = le.inverse_transform(y_pred)
body_ids = [str(body_inverse_index[body]) for body in val_all[:, 1]]
pred_for_cm = np.array([{'Headline': val_all[i, 0], 'Body ID': body_ids[i], 'Stance': predicted[i]} for i in range(len(val_all))])
gold_for_cm = np.array([{'Headline': val_all[i, 0], 'Body ID': body_ids[i], 'Stance': val_all[i, 2]} for i in range(len(val_all))])

In [ ]:
# Score using scorer.py:
test_score, cm = score_submission(gold_for_cm, pred_for_cm)
null_score, max_score = score_defaults(gold_for_cm)
print_confusion_matrix(cm)
print(SCORE_REPORT.format(max_score, null_score, test_score))

In [ ]:
# save to .csv in the correct format (see data/scorer.py and data/train_stances.csv)
pred_tosave = np.array([('Headline','Body ID','Stance')] + [(val_all[i, 0], body_ids[i], predicted[i]) for i in range(len(val_all))])
gold_tosave = np.array([('Headline','Body ID','Stance')] + [(val_all[i, 0], body_ids[i], val_all[i, 2]) for i in range(len(val_all))])

# TODO does not work ATM (UnicodeDecodeError) - try 'python scorer.py val_gold.csv val_predicted.csv' in data folder
np.savetxt('data/val_predicted.csv', pred_tosave, delimiter=',', fmt='%s', encoding='utf-8', newline='\n')
np.savetxt('data/val_gold.csv', gold_tosave, delimiter=',', fmt='%s', encoding='utf-8', newline='\n')

In [ ]:
# TODO analyse importance of each feature

In [ ]:
GradientBoostingClassifier(n_estimators=200)
ACCURACY: 0.834  ||  1604.25  ||  No Added Features (just Cosine and KL-Divergence)
ACCURACY: 0.861  ||  1743.5   ||  'Raw' Unigram Match
ACCURACY: 0.861  ||  1743.5   ||  1+log(x) Unigram Match
ACCURACY: 0.860  ||  1743.5   ||  1-1/e^(x/2) Unigram Match
ACCURACY: 0.884  ||  1834.5   ||  x/len(body) Unigram Match
ACCURACY: 0.873  ||  1790.5   ||  log(x)/log(len(body)) Unigram Match
ACCURACY: 0.884  ||  1834.25  ||  (x/len(body))^1/e Unigram Match
    
LogisticRegression(C=1e5)
ACCURACY: 0.880  ||  1795.5   ||  (x/len(body))^1/e Unigram Match

LogisticRegression(C=1e2, multi_class='multinomial', solver='saga')
ACCURACY: 0.881  ||  1818.5   ||  (x/len(body))^1/e Unigram Match
ACCURACY: 0.882  ||  1819.5   ||  (x/len(body))^1/e 2-gram Match
ACCURACY: 0.882  ||  1820.5   ||  (x/len(body))^1/e 3-gram Match